# Chapter 38: Debugging TypeScript

Debugging is an essential skill for developers, and TypeScript adds a layer of complexity since the code you write (TypeScript) is different from the code that executes (JavaScript). Source maps bridge this gap, allowing you to debug your original TypeScript source code even when the runtime is executing compiled JavaScript. This chapter covers comprehensive debugging strategies for TypeScript applications across different environments.

---

## 38.1 Source Maps Configuration and Generation

Source maps are files that map the compiled JavaScript back to the original TypeScript source, enabling breakpoint debugging in the original source files.

### Understanding Source Maps

When TypeScript compiles your code, it can generate `.js.map` files that contain mapping information:

```json
// tsconfig.json
{
  "compilerOptions": {
    "sourceMap": true,           // Generate .js.map files
    "inlineSourceMap": false,    // Embed map in .js file (alternative)
    "inlineSources": false,      // Embed TS source in map
    "sourceRoot": "/src",        // Root path for debugger
    "mapRoot": "/maps",          // Location to emit maps
    "outFile": "bundle.js",      // Single output file
    "outDir": "./dist"           // Output directory
  }
}
```

**Explanation:**
- `sourceMap: true`: Creates separate `.js.map` files alongside compiled JavaScript
- `inlineSourceMap`: Embeds the map as a base64 string in the JavaScript file (useful for libraries, not for applications)
- `inlineSources`: Embeds the original TypeScript source code in the map file (allows debugging without original source files)
- `sourceRoot`: Specifies the root path that the debugger should use to resolve source files

### Source Map Strategies

**For Development:**
```json
{
  "compilerOptions": {
    "sourceMap": true,
    "inlineSources": true,
    "sourceRoot": "/src"
  }
}
```

**For Production (Debuggable but Secure):**
```json
{
  "compilerOptions": {
    "sourceMap": true,
    "sourceRoot": "/src"
  },
  "exclude": ["**/*.test.ts", "**/*.spec.ts"]
}
```

**For Production (No Source Maps):**
```json
{
  "compilerOptions": {
    "sourceMap": false
  }
}
```

**Webpack/Vite Source Map Configuration:**

```javascript
// vite.config.ts
export default defineConfig({
  build: {
    sourcemap: true,        // Generate source maps
    sourcemapExcludeSources: false  // Include source content
  }
});

// webpack.config.js
module.exports = {
  devtool: 'source-map',    // High-quality source maps
  // Alternatives:
  // 'eval-source-map' - Faster, for development
  // 'cheap-module-source-map' - No column info, faster
  // 'hidden-source-map' - No reference in bundle, for production
};
```

---

## 38.2 Debugging in VS Code

VS Code provides excellent TypeScript debugging support through `launch.json` configurations.

### Node.js Debugging Configuration

**.vscode/launch.json:**

```json
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Debug Current TS File",
      "type": "node",
      "request": "launch",
      "runtimeExecutable": "npx",
      "runtimeArgs": ["tsx"],
      "args": ["${relativeFile}"],
      "env": {
        "NODE_ENV": "development"
      },
      "console": "integratedTerminal",
      "skipFiles": ["<node_internals>/**"],
      "resolveSourceMapLocations": [
        "${workspaceFolder}/**",
        "!**/node_modules/**"
      ]
    },
    {
      "name": "Debug Compiled JS",
      "type": "node",
      "request": "launch",
      "program": "${workspaceFolder}/dist/index.js",
      "preLaunchTask": "npm: build",
      "sourceMaps": true,
      "outFiles": ["${workspaceFolder}/dist/**/*.js"],
      "resolveSourceMapLocations": [
        "${workspaceFolder}/dist/**/*.js",
        "${workspaceFolder}/dist/**/*.js.map"
      ],
      "env": {
        "NODE_ENV": "development"
      },
      "console": "integratedTerminal"
    },
    {
      "name": "Attach to Process",
      "type": "node",
      "request": "attach",
      "port": 9229,
      "restart": true,
      "sourceMaps": true,
      "outFiles": ["${workspaceFolder}/dist/**/*.js"],
      "localRoot": "${workspaceFolder}",
      "remoteRoot": "/app"
    },
    {
      "name": "Debug Tests",
      "type": "node",
      "request": "launch",
      "runtimeExecutable": "npx",
      "runtimeArgs": [
        "vitest",
        "run",
        "${relativeFile}"
      ],
      "console": "integratedTerminal",
      "internalConsoleOptions": "neverOpen",
      "env": {
        "NODE_ENV": "test"
      }
    }
  ]
}
```

**Explanation:**
- **Debug Current TS File**: Uses `tsx` to run TypeScript directly without pre-compiling
- **Debug Compiled JS**: Runs compiled JavaScript with source maps pointing back to TypeScript
- **Attach to Process**: Connects to a running Node.js process (useful for Docker or long-running servers)
- **Debug Tests**: Runs Vitest on the current file with debugger attached

### Frontend Debugging (Chrome/Edge)

```json
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Debug Frontend (Chrome)",
      "type": "chrome",
      "request": "launch",
      "url": "http://localhost:5173",
      "webRoot": "${workspaceFolder}/src",
      "sourceMaps": true,
      "sourceMapPathOverrides": {
        "webpack:///src/*": "${webRoot}/*",
        "webpack:///./*": "${webRoot}/*",
        "webpack:///./~/*": "${webRoot}/node_modules/*"
      },
      "skipFiles": [
        "<node_internals>/**",
        "${workspaceFolder}/node_modules/**"
      ],
      "smartStep": true,
      "runtimeArgs": [
        "--remote-debugging-port=9222"
      ]
    },
    {
      "name": "Debug Frontend (Attach)",
      "type": "chrome",
      "request": "attach",
      "port": 9222,
      "webRoot": "${workspaceFolder}/src",
      "sourceMaps": true,
      "sourceMapPathOverrides": {
        "vite:///src/*": "${webRoot}/*"
      }
    }
  ]
}
```

**VS Code Debugging Tips:**

```typescript
// Use debugger statement to force breakpoint
function complexCalculation(data: Data) {
  debugger; // Execution will pause here when debugger is attached
  
  const result = processData(data);
  
  // Conditional debugger
  if (result.error) {
    debugger; // Only breaks when there's an error
  }
  
  return result;
}

// Logpoints (VS Code feature - no code change needed)
// Right-click gutter → "Add Logpoint" → Type: "Processing item {item.id}"
```

---

## 38.3 Chrome DevTools Debugging

Browser debugging allows inspection of React/Vue/Angular applications with full TypeScript source visibility.

### Setting Up Chrome DevTools

**1. Source Maps in Development:**
Vite and Webpack dev servers automatically serve source maps. Ensure your `vite.config.ts` has:
```typescript
export default defineConfig({
  build: {
    sourcemap: true
  },
  esbuild: {
    sourcemap: true,
    target: 'es2020'
  }
});
```

**2. TypeScript in DevTools:**
- Open Chrome DevTools (F12 or Cmd+Option+I)
- Go to Sources tab
- Look for `webpack://` or `vite://` in the file tree
- Original `.ts` and `.tsx` files appear here with full debugging support

### Advanced DevTools Techniques

**Conditional Breakpoints:**
Right-click a line number → "Add conditional breakpoint":
```typescript
// Break only when user is admin and count > 100
user.role === 'admin' && count > 100
```

**XHR/Fetch Breakpoints:**
- DevTools → Sources tab → XHR/fetch Breakpoints (right sidebar)
- Click + to add URL pattern: `api/users`
- Execution pauses when any request matching the pattern is made

**Logpoints:**
Instead of `console.log`, right-click line → "Add logpoint":
```
User clicked at position: {x}, {y}, Component state: {this.state}
```

**Blackboxing:**
Ignore library code when stepping:
- DevTools Settings → Ignore List → Add pattern: `/node_modules/`
- Or right-click a file in Sources → "Blackbox script"

---

## 38.4 Node.js Debugger and Inspect Protocol

Node.js has a built-in debugger that works with the Chrome DevTools Protocol.

### Starting Node with Inspector

```bash
# Method 1: Command line flag
node --inspect-brk dist/index.js

# Method 2: With tsx (TypeScript execution)
npx tsx --inspect-brk src/index.ts

# Method 3: Attach to running process
node --inspect dist/index.js
# Then attach via chrome://inspect or VS Code

# Method 4: Specific port and host
node --inspect=0.0.0.0:9229 dist/index.js
```

**Programmatic Debugging:**

```typescript
// Enable inspector programmatically
import inspector from 'inspector';

if (process.env.NODE_ENV === 'development') {
  inspector.open(9229, '0.0.0.0', true); // port, host, wait
}

// Set breakpoint programmatically
inspector.Session.prototype.post('Debugger.setBreakpointByUrl', {
  lineNumber: 10,
  url: 'file:///app/src/server.ts',
  columnNumber: 0
});
```

### Debugging Express Applications

```typescript
// server.ts with debugging hooks
import express from 'express';

const app = express();

// Debug middleware to inspect requests
if (process.env.DEBUG === 'true') {
  app.use((req, res, next) => {
    debugger; // Break here to inspect req object
    console.log(`[DEBUG] ${req.method} ${req.path}`);
    next();
  });
}

app.get('/api/users/:id', async (req, res) => {
  const userId = req.params.id;
  
  // Set breakpoint here to debug specific route
  debugger;
  
  try {
    const user = await getUser(userId);
    res.json(user);
  } catch (error) {
    debugger; // Inspect error details
    res.status(500).json({ error: error.message });
  }
});

// Graceful shutdown for debugging
process.on('SIGUSR1', () => {
  console.log('Received SIGUSR1, opening inspector...');
  require('inspector').open(9229);
});
```

---

## 38.5 Debugging Tests

Setting breakpoints in tests allows inspection of test state and assertion failures.

### Vitest Debugging

```json
// .vscode/launch.json for Vitest
{
  "type": "node",
  "request": "launch",
  "name": "Debug Vitest",
  "program": "${workspaceFolder}/node_modules/vitest/vitest.mjs",
  "args": ["run", "--reporter", "verbose", "${relativeFile}"],
  "console": "integratedTerminal",
  "internalConsoleOptions": "neverOpen",
  "env": {
    "NODE_ENV": "test"
  }
}
```

**In-Test Debugging:**

```typescript
// user.service.test.ts
import { describe, it, expect } from 'vitest';
import { UserService } from './user.service';

describe('UserService', () => {
  it('should create user with valid data', async () => {
    const service = new UserService();
    
    // Set breakpoint here
    debugger;
    
    const result = await service.create({
      name: 'John',
      email: 'john@example.com'
    });
    
    // Inspect result before assertion
    debugger;
    
    expect(result.id).toBeDefined();
    expect(result.name).toBe('John');
  });
});
```

### Jest Debugging

```json
// .vscode/launch.json for Jest
{
  "type": "node",
  "request": "launch",
  "name": "Debug Jest Tests",
  "runtimeExecutable": "npx",
  "runtimeArgs": [
    "jest",
    "--runInBand",
    "--testTimeout=999999",
    "--no-cache",
    "${relativeFile}"
  ],
  "console": "integratedTerminal",
  "internalConsoleOptions": "neverOpen",
  "env": {
    "NODE_ENV": "test"
  }
}
```

**Key Jest Debugging Flags:**
- `--runInBand`: Run tests serially (required for debugging)
- `--testTimeout=999999`: Prevent timeout while stepping through code
- `--no-cache`: Ensure fresh compilation

---

## 38.6 Advanced Debugging Techniques

### Conditional Breakpoints

Set breakpoints that only trigger under specific conditions:

```typescript
// In VS Code or Chrome DevTools, right-click line and set condition:
// items.length > 100 && user.role !== 'admin'

function processLargeDataset(items: Data[], user: User) {
  items.forEach((item, index) => {
    // Breakpoint condition: index === 50 || item.id === 'special-id'
    processItem(item);
  });
}
```

### Hit Count Breakpoints

Break only after a line has been executed N times:

```typescript
// Breakpoint settings: Hit count = 50
// Useful for debugging loop iterations

for (let i = 0; i < 1000; i++) {
  processData(i); // Break only on 50th iteration
}
```

### Watcher Expressions

Monitor expressions while stepping through code:

```
// Add to Watch panel in VS Code or Chrome DevTools:
user.permissions.includes('write')
JSON.stringify(config)
new Date().toISOString()
data.filter(x => x.active).length
```

### Async Stack Traces

Enable async stack traces for better debugging of promises:

```json
// tsconfig.json
{
  "compilerOptions": {
    "target": "ES2022",
    "module": "NodeNext"
  }
}
```

```bash
# Node.js flag for async stack traces
node --async-stack-traces dist/index.js
```

### Debugging Decorators

```typescript
// Debug decorator execution
function Debug(target: any, propertyKey: string, descriptor: PropertyDescriptor) {
  const originalMethod = descriptor.value;
  
  descriptor.value = function(...args: any[]) {
    debugger; // Break when decorated method is called
    console.log(`Calling ${propertyKey} with`, args);
    
    const result = originalMethod.apply(this, args);
    
    debugger; // Break after method execution
    console.log(`Result:`, result);
    
    return result;
  };
  
  return descriptor;
}

class Service {
  @Debug
  async fetchData(id: string) {
    return await api.get(`/data/${id}`);
  }
}
```

---

## 38.7 Remote Debugging

Debug applications running on servers, containers, or other machines.

### SSH Tunneling for Remote Debug

```bash
# On local machine, forward remote debugging port
ssh -L 9229:localhost:9229 user@remote-server

# Now attach VS Code to localhost:9229 to debug remote process
```

### Docker Debugging

**Dockerfile with debug support:**

```dockerfile
# Development stage
FROM node:20-alpine AS development

WORKDIR /app

# Install dependencies
COPY package*.json ./
RUN npm ci

# Copy source
COPY . .

# Expose app port and debug port
EXPOSE 3000 9229

# Start with inspector
CMD ["npx", "tsx", "watch", "--inspect=0.0.0.0:9229", "src/index.ts"]
```

**docker-compose.debug.yml:**

```yaml
version: '3.8'

services:
  app:
    build:
      context: .
      target: development
    ports:
      - "3000:3000"
      - "9229:9229"
    environment:
      - NODE_ENV=development
      - DEBUG=true
    volumes:
      - .:/app
      - /app/node_modules
```

**VS Code Launch Configuration for Docker:**

```json
{
  "version": "0.2.0",
  "configurations": [
    {
      "name": "Attach to Docker",
      "type": "node",
      "request": "attach",
      "port": 9229,
      "address": "localhost",
      "localRoot": "${workspaceFolder}",
      "remoteRoot": "/app",
      "sourceMaps": true,
      "outFiles": ["${workspaceFolder}/dist/**/*.js"],
      "skipFiles": ["<node_internals>/**"]
    }
  ]
}
```

### Kubernetes Debugging

```yaml
# k8s-debug.yaml
apiVersion: v1
kind: Pod
metadata:
  name: debug-pod
spec:
  containers:
  - name: app
    image: myapp:latest
    command: ["node", "--inspect=0.0.0.0:9229", "dist/index.js"]
    ports:
    - containerPort: 3000
    - containerPort: 9229
---
apiVersion: v1
kind: Service
metadata:
  name: debug-service
spec:
  selector:
    app: debug-pod
  ports:
  - port: 9229
    targetPort: 9229
    name: debug
  type: LoadBalancer
```

**Port Forward for K8s:**
```bash
kubectl port-forward pod/debug-pod 9229:9229
```

---

## 38.8 Debugging Common TypeScript Issues

### "Cannot read property of undefined" in Mapped Types

```typescript
// Issue: Runtime error despite TypeScript saying it's safe
interface Config {
  database?: {
    host: string;
    port: number;
  };
}

const config: Config = {};

// TypeScript might not catch this if config comes from JSON
debugger; // Inspect config here
console.log(config.database.host); // Runtime error!

// Fix: Optional chaining with debugger fallback
console.log(config.database?.host ?? 'localhost');
```

### Async/Await Stack Traces

```typescript
// Issue: Lost stack trace in async errors
async function fetchUser(id: string) {
  debugger; // Check call stack
  try {
    return await db.query(`SELECT * FROM users WHERE id = ${id}`);
  } catch (error) {
    debugger; // Inspect error stack
    throw new Error(`Failed to fetch user: ${error.message}`);
  }
}
```

### Generic Type Debugging

```typescript
// Debug generic type resolution
function processData<T extends { id: string }>(items: T[]): T[] {
  debugger; // Hover over T in VS Code to see resolved type
  
  return items.filter(item => {
    debugger; // Check item type here
    return item.id !== null;
  });
}

// Usage - T resolves to User
interface User { id: string; name: string; }
const users = processData<User>([{ id: '1', name: 'John' }]);
```

---

## 38.9 Chapter Summary and Exercises

### Chapter Summary

This chapter covered debugging strategies for TypeScript:

**Key Concepts:**

1. **Source Maps**: Bridge between TypeScript source and compiled JavaScript. Configure via `tsconfig.json` (`sourceMap: true`). Essential for debugging original source code.

2. **VS Code Debugging**: `launch.json` configurations for Node.js (direct tsx or compiled), Chrome (frontend), and test runners. Key settings: `sourceMaps: true`, `outFiles`, and `resolveSourceMapLocations`.

3. **Chrome DevTools**: Browser debugging for frontend apps. Use `webpack://` or `vite://` sources. Features: conditional breakpoints, XHR breakpoints, logpoints, and blackboxing.

4. **Node.js Inspector**: `--inspect` and `--inspect-brk` flags enable debugging protocol. Attach via Chrome (`chrome://inspect`) or VS Code. Useful for production debugging.

5. **Test Debugging**: Configure Jest/Vitest with `--runInBand` and increased timeout. Set breakpoints in test files to inspect setup and assertions.

6. **Advanced Techniques**: Conditional breakpoints (expressions), hit count breakpoints, watcher expressions, and async stack traces for promise chains.

7. **Remote/Docker**: Port forwarding and volume mapping enable debugging containerized applications. Map local source to remote container paths.

### Practical Exercises

**Exercise 1: Source Map Configuration**

Configure a project with:
- Separate source map files for development
- Inline source maps for a library build
- Verify breakpoints work in both VS Code and Chrome DevTools
- Disable source maps for production build and verify bundle size difference

**Exercise 2: VS Code Debugging Setup**

Create `launch.json` configurations for:
- Debugging a TypeScript Express API with tsx
- Debugging a React + Vite frontend
- Debugging Jest tests with breakpoints
- Attaching to a running Node process on port 9229

**Exercise 3: Docker Debugging**

Set up a Docker Compose environment with:
- Node.js app running with `--inspect`
- Volume mapping for live reload
- VS Code attach configuration for Docker
- Demonstrate setting a breakpoint in container that stops local VS Code

**Exercise 4: Production Debugging**

Simulate a production debugging scenario:
- Build application with source maps but don't deploy them
- Deploy only code, keep maps in S3/private storage
- When error occurs, download source map and use `source-map` library to resolve stack trace
- Or use error tracking service (Sentry) with source map upload

**Exercise 5: Advanced Breakpoints**

Write a function with:
- Conditional breakpoint that triggers only on specific data patterns
- Logpoint that outputs variable state without code changes
- Watch expression monitoring derived state
- Demonstrate stepping through async/await code maintaining context

### Additional Resources

- **VS Code Debugging**: https://code.visualstudio.com/docs/typescript/typescript-debugging
- **Chrome DevTools**: https://developer.chrome.com/docs/devtools/javascript/
- **Node.js Debugging**: https://nodejs.org/en/docs/guides/debugging-getting-started/
- **Source Maps Spec**: https://sourcemaps.info/spec.html

---

## Coming Up Next: Chapter 39 - TypeScript Compilation Performance

In the next chapter, we will explore optimizing TypeScript build times and performance:

- Understanding TypeScript compiler performance bottlenecks
- Project References for monorepos
- Incremental compilation strategies
- Optimizing tsconfig.json for speed
- Build tool comparisons (tsc, esbuild, swc, babel)
- Type checking vs transpilation separation
- Memory optimization for large projects
- CI/CD build optimization

As TypeScript projects grow, compilation performance becomes critical for developer productivity and deployment pipelines. Understanding how to optimize these builds while maintaining type safety is essential for scaling TypeScript adoption in large organizations.